In [1]:
from altaipony.lcio import from_mast, from_path
from altaipony.flarelc import FlareLightCurve
from altaipony.altai import find_iterative_median
from lightkurve import search_targetpixelfile

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd

import copy

def refine_detrended_flux_err(flcd, mask_pos_outliers_sigma=2.5, 
                              std_rolling_window_length=15, pad=25):
    """Attempt to recover a good estimate of the ligh curve noise.
    Start out from a simple standard deviation of the flux.
    Then filter out outliers above `mask_pos_outliers_sigma`.
    Apply rolling window standard deviation on the filtered array.
    Calculate a mean standard deviation from the result.
    Fill in this mean into the masked values.

    Parameters:
    -----------
    flcd : de-trended FlareLightCurve

    mask_pos_outliers_sigma : float
        sigma value above which to mask positive outliers
    std_rolling_window_length : int
        rolling window length for standard deviation calculation
    pad : int
        How many values to pad-mask around positive outliers.

    Return:
    --------
    FlareLightCurve with refined `detrended_flux_err` attribute.

    """

    # start with a first approximation to std
    flcd.detrended_flux_err[:] =  np.nanstd(flcd.detrended_flux)

    # and refine it:
    flcd = find_iterative_median(flcd)

    filtered = copy.deepcopy(flcd.detrended_flux)

    # mask strong positive outliers so that they don't add to std
    filtered[flcd.detrended_flux - flcd.it_med > mask_pos_outliers_sigma * flcd.detrended_flux_err] = np.nan

    # apply rolling window std
    flcd.detrended_flux_err[:] = pd.Series(filtered).rolling(std_rolling_window_length, min_periods=1).std()

    # set std to mean value if calculation fails to inf
    meanstd = np.nanmean(flcd.detrended_flux_err)

    # pad the excluded values not to create spikes of high error around flares
    isin = np.invert(np.isfinite(flcd.detrended_flux_err))
    x = np.where(isin)[0]
    for i in range(-pad, pad+1):
        y = x + i
        y[np.where(y > len(isin) - 1)] = len(isin) - 1
        isin[y] = True

    x = np.where(isin)[0]
    flcd.detrended_flux_err[x] = meanstd

    return flcd

import time

/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/importlib/_bootstrap_external.py:426: ImportWarning: Not importing directory /home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/site-packages/mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
/home/ekaterina/Documents/000_envs/forallpurposes/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [ ]:
df = pd.read_csv("../k2scoc/results/tables/improved_flares.csv",skiprows=1,
                 names = ["istart", "istop", 
                         "cstart", "cstop","tstart", "tstop",
                          "ed_rec", "ed_rec_err","ampl_rec","tot_dtp",
                          "EPIC", "C"
                         ])
df=df.drop_duplicates().sort_values(by="EPIC")
df.shape

In [ ]:
clus = pd.read_csv("joblistonlyk2sc.sh",delimiter=" ",
                 names = ["python","call","EPIC","channel","C","cluster"
                         ])

In [ ]:
df2 = df.merge(clus[["EPIC","C","cluster"]], how="left", on=["EPIC","C"])
np.where(df2["cluster"].isnull()), df2.shape

In [ ]:
df2.to_csv("../k2scoc/results/tables/improved_flares_vetted.csv",index=False)

In [ ]:
df.groupby(["EPIC","C"]).count().shape

In [ ]:
df = pd.read_csv("../k2scoc/results/tables/improved_flares_vetted.csv")
df=df.drop_duplicates().sort_values(by="EPIC")
df.shape

In [ ]:
df = pd.read_csv("../k2scoc/results/tables/full_table.csv")
df["complex"]=0
df.loc[(df.real==1) & (df.note == "complex!") &(~df.note.isnull()), "complex"] =1
df.note[df.note=="complex!"]=""

In [ ]:
df.to_csv("../k2scoc/results/tables/full_table.csv", index=False)

In [ ]:
df2 = df[(df.real==1) & (df.note == "complex!") &(~df.note.isnull())]#
df2.sort_values(by=["ed_rec"], ascending=False)[["cstart","cstop","EPIC","C","note","ed_rec","cluster"]].head(3)

In [ ]:
EPIC = 211104176

C = 4
subdf = df[(df.C==C) & (df.EPIC==EPIC) & (df.istop - df.istart > 2)]

flc = from_path(f"../k2scoc/pony_k2sc_k2_llc_{EPIC}-c{C:02d}_kepler_v2_lc.fits", mode="AltaiPony", mission="K2")

In [ ]:

for i, r in subdf.iterrows():
    plt.figure(figsize=(7,5))
    plt.plot(flc.time, flc.detrended_flux,c="k")
    plt.plot(flc.time, flc.flux,c="b", alpha=.3)
    sta, sto = int(r.istart), int(r.istop)+1
    plt.scatter(flc.time[sta:sto], flc.detrended_flux[sta:sto], c="r", label=r.cstart)#, label=f"EPIC {EPIC}, C{C}")#
    plt.xlim(r.tstart-5,r.tstop+5)
    plt.ylim(np.min(1.*flc.detrended_flux),1.*np.max(flc.detrended_flux))
    plt.legend()
   # plt.savefig(f"plots/EPIC_{EPIC}_C{C}_{r.istart}_multiperiod.png",dpi=300)
    #print(r.istart)

## View also TPF

In [ ]:
df2.shape

In [8]:
EPIC = 219766926





C = 7
tpf = search_targetpixelfile("EPIC {}".format(EPIC), campaign=C).download()


15% (594/4043) of the cadences will be ignored due to the quality mask (quality_bitmask=1130799).
15% (594/4043) of the cadences will be ignored due to the quality mask (quality_bitmask=1130799).


In [9]:
#tpf = tpf[~np.isnan(tpf.flux)]
tpf.interact(notebook_url="http://localhost:8888")


INFO:bokeh.server.server:Starting Bokeh server version 2.0.0 (running on Tornado 6.0.4)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


200 GET /autoload.js?bokeh-autoload-element=13042&bokeh-absolute-url=http://localhost:34393&resources=none (127.0.0.1) 253.51ms
101 GET /ws (127.0.0.1) 0.95ms
INFO:bokeh.server.views.ws:WebSocket connection opened
INFO:bokeh.server.views.ws:ServerConnection created


In [ ]:
# import seaborn as sns
# from matplotlib.colors import PowerNorm
# %matplotlib inline
# ras = tpf.get_coordinates(cadence=0)[0]
# decs = tpf.get_coordinates(cadence=0)[1]
# labels = np.chararray(ras.shape, itemsize=20, unicode=True)

# for i, row in enumerate(ras):
#     if i%5==0:
#         for j, col in enumerate(row):
#             if j%5 ==0:
#         #         l = "{:.4f}\n{:.4f}".format(col-tpf.ra, decs[i][j]-tpf.dec)
#                # l = "{:.4f}\n{:.4f}".format(col-92.154527, decs[i][j]-24.233511)
#                 l = "{:.03f}".format(100*np.sqrt((col-92.154527)**2+(decs[i][j]-24.233511)**2))
#                 labels[i][j] = l

# fig, ax = plt.subplots(figsize=(10,8))
# sns.heatmap(tpf.flux[0][:][::-1], annot = labels[:][::-1], fmt = '', ax=ax, cmap="viridis", norm=PowerNorm(gamma=1./2.))

In [ ]:
250./15000


In [ ]:
5483/(1300/4.5)

In [ ]:
1.04875383187081e32*60
